In [1]:
!pip install xgboost

In [2]:
import pandas as pd
from sklearn import linear_model, metrics, preprocessing, tree, model_selection, feature_selection, ensemble
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb

### Loading training and tesing data

In [4]:
train_set = pd.read_csv('train.csv')
test_set = pd.read_csv('test.csv')

### Data Processing & EDA

In [5]:
train_set.head()

,created_at,default_profile,default_profile_image,description,favourites_count,followers_count,friends_count,geo_enabled,id,lang,location,profile_background_image_url,profile_image_url,screen_name,statuses_count,verified,average_tweets_per_day,account_age_days,target
0,2012-01-15 23:40:09,True,False,Cosplayer/Fitness lover. Come to me https://t....,74,7,0,False,465096524,en,unknown,http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/9666745212...,reml5477,20,False,0.006,3138,1
1,2016-10-04 00:44:39,False,False,pobody’s nerfect,50443,164,590,True,783105517673648132,cy,she/her,http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/1281752126...,kinlibra,6469,False,4.572,1415,0
2,2009-05-23 04:04:13,False,False,gracias por participar 🏅,9394,208,189,False,41970759,es,La diaspora,http://abs.twimg.com/images/themes/theme17/bg.gif,http://pbs.twimg.com/profile_images/1233811596...,_delaualau,30296,False,7.378,4106,0
3,2009-05-17 04:31:31,False,False,Stand Up Comedian/Actor from North Philadelphi...,46,66180,1090,True,40607946,en,"Calabasas, CA",http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/1184851104...,SpankHorton,164957,False,40.116,4112,0
4,2009-02-16 13:11:21,True,False,Assignment Editor at NBC10 and President of Ja...,1223,487,867,True,20983433,en,"Jenkintown, PA",http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/5234863934...,javelinjt,1752,False,0.417,4201,0


In [6]:
test_set.head()

,index,created_at,default_profile,default_profile_image,description,favourites_count,followers_count,friends_count,geo_enabled,id,lang,location,profile_background_image_url,profile_image_url,screen_name,statuses_count,verified,average_tweets_per_day,account_age_days
0,0,2007-03-27 22:56:04,False,False,Dad. #ADHD @fasternormal Pod host. Author/keyn...,22290,163813,1746,True,2569261,en,"New York, NY",http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/1186468889...,petershankman,42013,True,8.586,4893
1,1,2013-03-30 08:47:01,True,False,NaN,685,44,637,False,1316038452,NaN,unknown,http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/9637876928...,Mihail_HD,111,False,0.041,2699
2,2,2009-04-25 15:24:08,False,False,#UGAMOM #GODAWGS,12257,276,194,True,35233977,de,"Augusta, GA",http://abs.twimg.com/images/themes/theme3/bg.gif,http://pbs.twimg.com/profile_images/1246470377...,TinkerMazell,18142,False,4.390,4133
3,3,2010-08-06 22:21:56,True,False,"golfer, traveler, car guy, PCA member, owns 19...",30806,1124,4999,True,175542625,en,New York,http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/1391357162...,joecangolfICC,20624,False,5.627,3665
4,4,2015-03-02 12:29:39,True,False,Do you like it gently? Come in! 💚💚💚 https://t....,224,22,265,False,3056068536,en,unknown,http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/9680856906...,TiesiogDovydas,64,False,0.032,1996


In [7]:
print("Size of Training Instances:", train_set.shape[0])
print("Sice of Features:", train_set.shape[1])
print(train_set.info())

print("Size of Testing Instances:", test_set.shape[0])
print("Sice of Features:", test_set.shape[1])
print(test_set.info())

Size of Training Instances: 26206
Sice of Features: 19
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26206 entries, 0 to 26205
Data columns (total 19 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   created_at                    26206 non-null  object 
 1   default_profile               26206 non-null  bool   
 2   default_profile_image         26206 non-null  bool   
 3   description                   21115 non-null  object 
 4   favourites_count              26206 non-null  int64  
 5   followers_count               26206 non-null  int64  
 6   friends_count                 26206 non-null  int64  
 7   geo_enabled                   26206 non-null  bool   
 8   id                            26206 non-null  int64  
 9   lang                          20618 non-null  object 
 10  location                      26204 non-null  object 
 11  profile_background_image_url  22971 non-null  object 
 12  profi

1. Domain Knowledge Processing: there are some configuration features should be set by users manually, which requires users to put effor in them. Setting these features may reflect how importantly users treat their accounts and may distinguish genius accounts from others. Therefore, these features can divided the users into binary groups - setting (1) and not setting (0).  
    These features are *description*, *geo_enabled*, *lang*, *location*, *verified*.  

2. Redundant Features - *create_at*, *id*, *profile_background_image_url*, *profile_image_url* and *screen_name*. 
Every user is expected to have crate time, id, and screen name no matter whether using default profile settings.   
Moreover, in the training dataset, it cannot distinguish users by profile background image url and profile image url. In addition, whether using defualt profile can reflect whether user put effort in profile configuration.   
Therefore, these features cannot distinguish the users and can be discarded.  

In [8]:
train_set.drop(columns=['created_at','id', 'profile_background_image_url', 'profile_image_url', 'screen_name'], inplace=True)
test_set.drop(columns=['created_at','id', 'profile_background_image_url', 'profile_image_url', 'screen_name'], inplace=True)

3. Missing Values:  
    a. description: missing values means user does not input his or her description, that is 0.  
    b. lang: missing values means user does not set his or her preferred lanaguages, that is 0.  
    c. location: missing values user does not provide his or her locaiton information, that is 0.

In [9]:
train_set['description'] = train_set['description'].isna().astype(int)
train_set['description'] = np.where(train_set['description'] == 1, 0, 1)

train_set['lang'] = train_set['lang'].isna().astype(int)
train_set['lang'] = np.where(train_set['lang'] == 1, 0, 1)

train_set['location'] = train_set['location'].isna().astype(int)
train_set['location'] = np.where(train_set['location'] == 1, 0, 1)

test_set['description'] = test_set['description'].isna().astype(int)
test_set['description'] = np.where(test_set['description'] == 1, 0, 1)

test_set['lang'] = test_set['lang'].isna().astype(int)
test_set['lang'] = np.where(test_set['lang'] == 1, 0, 1)

test_set['location'] = test_set['location'].isna().astype(int)
test_set['location'] = np.where(test_set['location'] == 1, 0, 1)

4. Convert Boolean into Binary INT

In [10]:
train_set['default_profile'] = train_set['default_profile'].astype(int)
train_set['default_profile_image'] = train_set['default_profile_image'].astype(int)
train_set['geo_enabled'] = train_set['geo_enabled'].astype(int)
train_set['verified'] = train_set['verified'].astype(int)

test_set['default_profile'] = test_set['default_profile'].astype(int)
test_set['default_profile_image'] = test_set['default_profile_image'].astype(int)
test_set['geo_enabled'] = test_set['geo_enabled'].astype(int)
test_set['verified'] = test_set['verified'].astype(int)

5. Feature scaling  
The ranges for favourites_count, follower_count, friends_count and statuses_count, averages_tweets_per_day, and account_age_days are very different, which may impact models due to different ranges. Therefore, scaling should be conducted on these features.

In [11]:
scaler = preprocessing.MinMaxScaler()
train_set[['followers_count', 'friends_count', 'favourites_count', 'statuses_count', 'average_tweets_per_day', 'account_age_days']] = scaler.fit_transform(train_set[['followers_count', 'friends_count', 'favourites_count', 'statuses_count', 'average_tweets_per_day', 'account_age_days']])
test_set[['followers_count', 'friends_count', 'favourites_count', 'statuses_count', 'average_tweets_per_day', 'account_age_days']] = scaler.transform(test_set[['followers_count', 'friends_count', 'favourites_count', 'statuses_count', 'average_tweets_per_day', 'account_age_days']])

In [12]:
train_set

,default_profile,default_profile_image,description,favourites_count,followers_count,friends_count,geo_enabled,lang,location,statuses_count,verified,average_tweets_per_day,account_age_days,target
0,1,0,1,0.000088,5.754616e-08,0.000000,0,1,1,0.000007,0,0.000005,0.568766,1
1,0,0,1,0.060056,1.348224e-06,0.000359,1,1,1,0.002334,0,0.003838,0.199657,0
2,0,0,1,0.011184,1.709943e-06,0.000115,0,1,1,0.010930,0,0.006193,0.776135,0
3,0,0,1,0.000055,5.440579e-04,0.000664,1,1,1,0.059510,0,0.033675,0.777421,0
4,1,0,1,0.001456,4.003569e-06,0.000528,1,1,1,0.000632,0,0.000350,0.796487,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26201,1,0,0,0.085900,2.943075e-06,0.000254,1,0,1,0.003565,0,0.003046,0.479863,0
26202,1,0,0,0.016195,2.416939e-06,0.000127,1,0,1,0.001568,0,0.001599,0.385390,0
26203,0,0,1,0.002738,9.306037e-06,0.000499,1,1,1,0.018618,0,0.011737,0.687232,0
26204,0,0,1,0.156654,3.756942e-06,0.000218,1,1,1,0.008381,0,0.004729,0.779991,0


In [13]:
# x_train = train_set.drop(columns=['target'])
# y_train = train_set['target']

x_train, x_test, y_train, y_test = model_selection.train_test_split(train_set.drop(columns=['target']), train_set['target'], test_size=0.2, shuffle = True, random_state=42)

### Logistic Regression

In [14]:
logreg_cv_model = linear_model.LogisticRegressionCV(
    Cs=10,
    cv=10,
    penalty='l1',
    solver='liblinear',
    scoring='roc_auc',
    random_state=42,
    n_jobs=-1,
    max_iter=1000
)

In [15]:
logreg_cv_model.fit(x_train, y_train)

LogisticRegressionCV(cv=10, max_iter=1000, n_jobs=-1, penalty='l1',
                     random_state=42, scoring='roc_auc', solver='liblinear')

In [16]:
param_grid_logreg = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear']
}
grid_search_logreg = model_selection.GridSearchCV(
    linear_model.LogisticRegression(random_state=42, max_iter=1000),
    param_grid_logreg,
    cv=10,
    scoring='roc_auc',
    n_jobs=-1,
    verbose=1
)

In [17]:
grid_search_logreg.fit(x_train, y_train)
best_logreg = grid_search_logreg.best_estimator_

Fitting 10 folds for each of 12 candidates, totalling 120 fits


In [18]:
y_train_pred_proba = logreg_cv_model.predict_proba(x_train)[:, 1]
train_auc = metrics.roc_auc_score(y_train, y_train_pred_proba)
print("Train AUC:", train_auc)
y_test_pred_proba = logreg_cv_model.predict_proba(x_test)[:, 1]
test_auc = metrics.roc_auc_score(y_test, y_test_pred_proba)
print("Test AUC:", test_auc)

Train AUC: 0.8254802670676067
Test AUC: 0.8289852828555255


### Decision Tree

In [21]:
dt_model = tree.DecisionTreeClassifier(random_state=42)

In [22]:
param_grid_dt = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 5, 10, 15, 20,30,40],
    'min_samples_split': [2, 3, 5, 7, 10, 15],
    'min_samples_leaf': [1, 2, 3, 4, 5],
    'max_features': [None, 'sqrt', 'log2']
}

In [23]:
grid_search_dt = model_selection.GridSearchCV(estimator=dt_model,
                              param_grid=param_grid_dt,
                              cv=5, 
                              scoring='roc_auc',
                              n_jobs=-1, 
                              verbose=1
                             )

In [32]:
grid_search_dt.fit(x_train, y_train)

Fitting 5 folds for each of 1260 candidates, totalling 6300 fits


GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [None, 5, 10, 15, 20, 30, 40],
                         'max_features': [None, 'sqrt', 'log2'],
                         'min_samples_leaf': [1, 2, 3, 4, 5],
                         'min_samples_split': [2, 3, 5, 7, 10, 15]},
             scoring='roc_auc', verbose=1)

In [33]:
best_dt_model = grid_search_dt.best_estimator_

In [34]:
y_train_pred_proba_dt = best_dt_model.predict_proba(x_train)[:, 1]
train_auc_dt = metrics.roc_auc_score(y_train, y_train_pred_proba_dt)
print(f"Train AUC for Decision Tree: {train_auc_dt:.4f}")
y_test_pred_proba_dt = best_dt_model.predict_proba(x_test)[:, 1]
test_auc_dt = metrics.roc_auc_score(y_test, y_test_pred_proba_dt)
print(f"Test AUC for Decision Tree: {test_auc_dt:.4f}")

Train AUC for Decision Tree: 0.9515
Test AUC for Decision Tree: 0.9050


In [ ]:
feature_importances = best_dt_model.feature_importances_
feature_importances_df = pd.DataFrame({'feature': x_train.columns, 'importance': feature_importances})
print(feature_importances_df.sort_values(by='importance', ascending=False).head(10))

                   feature  importance
4          followers_count    0.403168
3         favourites_count    0.187233
11  average_tweets_per_day    0.102436
12        account_age_days    0.083662
5            friends_count    0.074960
9           statuses_count    0.069607
10                verified    0.056756
6              geo_enabled    0.011844
2              description    0.004410
0          default_profile    0.003158


In [ ]:
x_train_selected = x_train.loc[:, feature_importances > 0.05]
x_test_selected = x_test.loc[:, feature_importances > 0.05]

In [ ]:
grid_search_dt.fit(x_train_selected, y_train)

Fitting 5 folds for each of 1260 candidates, totalling 6300 fits


GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [None, 5, 10, 15, 20, 30, 40],
                         'max_features': [None, 'sqrt', 'log2'],
                         'min_samples_leaf': [1, 2, 3, 4, 5],
                         'min_samples_split': [2, 3, 5, 7, 10, 15]},
             scoring='roc_auc', verbose=1)

In [ ]:
best_dt_model = grid_search_dt.best_estimator_

In [ ]:
y_train_pred_proba_dt = best_dt_model.predict_proba(x_train_selected)[:, 1]
train_auc_dt = metrics.roc_auc_score(y_train, y_train_pred_proba_dt)
print(f"Train AUC for Decision Tree: {train_auc_dt:.4f}")
y_test_pred_proba_dt = best_dt_model.predict_proba(x_test_selected)[:, 1]
test_auc_dt = metrics.roc_auc_score(y_test, y_test_pred_proba_dt)
print(f"Test AUC for Decision Tree: {test_auc_dt:.4f}")

Train AUC for Decision Tree: 0.9476
Test AUC for Decision Tree: 0.9038


In [ ]:
y_test_pred_dt = best_dt_model.predict(test_set.iloc[:,1:])
pred_results_dt = pd.DataFrame({'index': test_set['index'], 'target': y_test_pred_dt})
pred_results_dt.to_csv('pred_results_dt.csv', index=False)

### Random Forest

In [36]:
rf_model = ensemble.RandomForestClassifier(
    random_state=42,
    class_weight='balanced_subsample'
)

In [37]:
param_grid_rf = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3,5,8],
    'min_samples_split': [10,20,30],
    'min_samples_leaf': [4, 8, 12],
    'max_features': ['sqrt', 'log2']
}

In [38]:
kfold = model_selection.StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

In [39]:
grid_search_rf = model_selection.GridSearchCV(estimator=rf_model,
                              param_grid=param_grid_rf,
                              cv=kfold,
                              scoring='roc_auc',
                              n_jobs=-1,
                              verbose=2
                             )

In [40]:
grid_search_rf.fit(x_train, y_train)

Fitting 10 folds for each of 162 candidates, totalling 1620 fits


GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=42, shuffle=True),
             estimator=RandomForestClassifier(class_weight='balanced_subsample',
                                              random_state=42),
             n_jobs=-1,
             param_grid={'max_depth': [3, 5, 8],
                         'max_features': ['sqrt', 'log2'],
                         'min_samples_leaf': [4, 8, 12],
                         'min_samples_split': [10, 20, 30],
                         'n_estimators': [100, 200, 300]},
             scoring='roc_auc', verbose=2)

In [41]:
best_rf_model = grid_search_rf.best_estimator_

In [42]:
y_train_pred_proba_rf = best_rf_model.predict_proba(x_train)[:, 1]
train_auc_rf = metrics.roc_auc_score(y_train, y_train_pred_proba_rf)
print(f"Train AUC for tuned Random Forest: {train_auc_rf:.4f}")
y_test_pred_proba_rf = best_rf_model.predict_proba(x_test)[:, 1]
test_auc_rf = metrics.roc_auc_score(y_test, y_test_pred_proba_rf)
print(f"Test AUC for tuned Random Forest: {test_auc_rf:.4f}")

Train AUC for tuned Random Forest: 0.9348
Test AUC for tuned Random Forest: 0.9184


In [43]:
feature_importances = best_rf_model.feature_importances_
feature_importances_df = pd.DataFrame({'feature': x_train.columns, 'importance': feature_importances})
print(feature_importances_df.sort_values(by='importance', ascending=False).head(10))

                   feature  importance
4          followers_count    0.244928
5            friends_count    0.186150
3         favourites_count    0.173396
9           statuses_count    0.089925
10                verified    0.083757
11  average_tweets_per_day    0.068401
6              geo_enabled    0.061313
12        account_age_days    0.048963
2              description    0.017176
0          default_profile    0.014140


In [44]:
x_train_selected = x_train.loc[:, feature_importances > 0.05]
x_test_selected = x_test.loc[:, feature_importances > 0.05]

In [45]:
grid_search_rf.fit(x_train_selected, y_train)

Fitting 10 folds for each of 162 candidates, totalling 1620 fits


GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=42, shuffle=True),
             estimator=RandomForestClassifier(class_weight='balanced_subsample',
                                              random_state=42),
             n_jobs=-1,
             param_grid={'max_depth': [3, 5, 8],
                         'max_features': ['sqrt', 'log2'],
                         'min_samples_leaf': [4, 8, 12],
                         'min_samples_split': [10, 20, 30],
                         'n_estimators': [100, 200, 300]},
             scoring='roc_auc', verbose=2)

In [46]:
best_rf_model = grid_search_rf.best_estimator_

In [47]:
y_train_pred_proba_rf = best_rf_model.predict_proba(x_train_selected)[:, 1]
train_auc_rf = metrics.roc_auc_score(y_train, y_train_pred_proba_rf)
print(f"Train AUC for tuned Random Forest: {train_auc_rf:.4f}")
y_test_pred_proba_rf = best_rf_model.predict_proba(x_test_selected)[:, 1]
test_auc_rf = metrics.roc_auc_score(y_test, y_test_pred_proba_rf)
print(f"Test AUC for tuned Random Forest: {test_auc_rf:.4f}")

Train AUC for tuned Random Forest: 0.9290
Test AUC for tuned Random Forest: 0.9133


### XGBoost

In [48]:
xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)  # Important:  Set eval_metric

In [53]:
param_grid_xgb = {
    'n_estimators': [150, 200, 250],
    'learning_rate': [0.001, 0.01, 0.02],
    'max_depth': [4, 5, 6],
    'subsample': [0.7, 0.8],
    'colsample_bytree': [0.7, 0.8],
    'min_child_weight': [1, 3],
    'gamma': [0, 0.1],
    'reg_alpha': [0, 0.01],
    'reg_lambda': [0, 0.01]
}

In [62]:
grid_search_xgb = model_selection.GridSearchCV(
    estimator=xgb_model,
    param_grid=param_grid_xgb,
    scoring='roc_auc',
    cv=5,
    n_jobs=-1,
    verbose=1
)

In [55]:
grid_search_xgb.fit(x_train, y_train)

Fitting 5 folds for each of 1728 candidates, totalling 8640 fits


c:\Users\ivanj\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\training.py:183: UserWarning: [10:42:26] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False,
                                     eval_metric='logloss', feature_types=None,
                                     feature_weights=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraint...
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None, ...),
             n_jobs=-1,
             param_grid={'colsample_bytree': [0.7, 0.8], 'gamma': [0, 0.1],
                         'learning_rate': [0.001, 0.01, 0.02],
                         'max_depth': [4, 5, 6], 'min_child_weight': [1, 3],
                         'n_estimators': [150, 200, 250],
                         'reg_alpha': [0, 0.01], 'reg_lambda': [0, 0.01],
                         'subsample': [0.7, 0.8]},
             scoring='roc_auc', verbose=1)

In [56]:
best_xgb_model = grid_search_xgb.best_estimator_
y_pred_train_proba_xgb = best_xgb_model.predict_proba(x_train)[:, 1]
train_auc_xgb = metrics.roc_auc_score(y_train, y_pred_train_proba_xgb)
print(f"Train AUC for tuned XGBoost: {train_auc_xgb:.4f}")
y_pred_test_proba_xgb = best_xgb_model.predict_proba(x_test)[:, 1]
test_auc_xgb = metrics.roc_auc_score(y_test, y_pred_test_proba_xgb)
print(f"Test AUC for tuned XGBoost: {test_auc_xgb:.4f}")

Train AUC for tuned XGBoost: 0.9555
Test AUC for tuned XGBoost: 0.9341


In [57]:
feature_importance = best_xgb_model.feature_importances_
feature_importances_df = pd.DataFrame({'feature': x_train.columns, 'importance': feature_importances})
print(feature_importances_df.sort_values(by='importance', ascending=False).head(10))

                   feature  importance
4          followers_count    0.244928
5            friends_count    0.186150
3         favourites_count    0.173396
9           statuses_count    0.089925
10                verified    0.083757
11  average_tweets_per_day    0.068401
6              geo_enabled    0.061313
12        account_age_days    0.048963
2              description    0.017176
0          default_profile    0.014140


In [58]:
x_train_selected = x_train.loc[:, feature_importances > 0.05]
x_test_selected = x_test.loc[:, feature_importances > 0.05]

In [59]:
grid_search_xgb.fit(x_train_selected, y_train)

Fitting 5 folds for each of 1728 candidates, totalling 8640 fits


c:\Users\ivanj\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\training.py:183: UserWarning: [10:50:29] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False,
                                     eval_metric='logloss', feature_types=None,
                                     feature_weights=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraint...
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None, ...),
             n_jobs=-1,
             param_grid={'colsample_bytree': [0.7, 0.8], 'gamma': [0, 0.1],
                         'learning_rate': [0.001, 0.01, 0.02],
                         'max_depth': [4, 5, 6], 'min_child_weight': [1, 3],
                         'n_estimators': [150, 200, 250],
                         'reg_alpha': [0, 0.01], 'reg_lambda': [0, 0.01],
                         'subsample': [0.7, 0.8]},
             scoring='roc_auc', verbose=1)

In [60]:
best_xgb_model = grid_search_xgb.best_estimator_
y_pred_train_proba_xgb = best_xgb_model.predict_proba(x_train_selected)[:, 1]
train_auc_xgb = metrics.roc_auc_score(y_train, y_pred_train_proba_xgb)
print(f"Train AUC for tuned XGBoost: {train_auc_xgb:.4f}")
y_pred_test_proba_xgb = best_xgb_model.predict_proba(x_test_selected)[:, 1]
test_auc_xgb = metrics.roc_auc_score(y_test, y_pred_test_proba_xgb)
print(f"Test AUC for tuned XGBoost: {test_auc_xgb:.4f}")

Train AUC for tuned XGBoost: 0.9425
Test AUC for tuned XGBoost: 0.9244


In [61]:
train_predictors = train_set.drop(columns=['target'])
train_target = train_set['target']

In [63]:
# Original XGBoost model
grid_search_xgb.fit(x_train, y_train)

Fitting 5 folds for each of 1728 candidates, totalling 8640 fits


c:\Users\ivanj\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\training.py:183: UserWarning: [10:59:25] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False,
                                     eval_metric='logloss', feature_types=None,
                                     feature_weights=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraint...
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None, ...),
             n_jobs=-1,
             param_grid={'colsample_bytree': [0.7, 0.8], 'gamma': [0, 0.1],
                         'learning_rate': [0.001, 0.01, 0.02],
                         'max_depth': [4, 5, 6], 'min_child_weight': [1, 3],
                         'n_estimators': [150, 200, 250],
                         'reg_alpha': [0, 0.01], 'reg_lambda': [0, 0.01],
                         'subsample': [0.7, 0.8]},
             scoring='roc_auc', verbose=1)

In [64]:
best_xgb_model = grid_search_xgb.best_estimator_
train_target_proba = best_xgb_model.predict_proba(train_predictors)[:, 1]
train_auc_xgb = metrics.roc_auc_score(train_target, train_target_proba)
print(f"Train AUC for tuned XGBoost: {train_auc_xgb:.4f}")

Train AUC for tuned XGBoost: 0.9512


In [67]:
test_set_predictors = test_set.drop(columns=['index'])
test_set_target_proba = best_xgb_model.predict_proba(test_set_predictors)[:, 1]

test_set_prob = pd.DataFrame({'index': test_set['index'], 'target': test_set_target_proba})
test_set_prob.to_csv('test_set_prob.csv', index=False)